## Code

In [10]:
%load_ext ipython_sparql_pandas

The ipython_sparql_pandas extension is already loaded. To reload it, use:
  %reload_ext ipython_sparql_pandas


In [11]:
from SPARQLWrapper import SPARQLWrapper, JSON
from random import randint
sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-Stanford')
sparql.setReturnFormat(JSON)

In [3]:
#from the query results extract a list that contains the counts after splitting on certain attributes
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']

def extractCountAndAttr(result):
    number = []
    listOfObj = []
    listOfPred= []
    #ret = result.queryAndConvert()
    for attr in result["results"]["bindings"]:
        listOfObj.append(attr['o'])
        listOfPred.append(attr['p'])
    return(listOfObj,listOfPred)

def generateQuestion(listOfPred, listOfObj, index, subject):
    flag = True
    if ('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>') in AttrHistory:
        return generateQuestion(listOfPred, listOfObj, index+1, subject)
        
    else:
        AttrHistory.append('<' + str(listOfPred[index]['value']) +'> <'+str(listOfObj[index]['value'])+ '>')
        
    #print(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} {listOfObj[index]["value"].split("/")[-1]}?' )
    
    try:
        listOfObj[index]['datatype']
    except:
        flag = False
        
    if flag is True:
        datatype = listOfObj[index]["datatype"].split("#")[-1]
        answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '"'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype)
    else:
        if len(listOfObj[index]["value"].split('/')) == 1:
            answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '"'+str(listOfObj[index]['value'])+ '"')
        else:
            answer = findAnswer(subject, '<' + str(listOfPred[index]['value']) +'>', '<'+str(listOfObj[index]['value'])+ '>')
    #print(answer)
    
    
    if answer:
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'.') 
        else:
            if len(listOfObj[index]["value"].split('/')) == 1:
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+ '".')  
            else:
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>.')    
        
    elif not answer:
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'. }}')
        else:
            if len(listOfObj[index]["value"].split('/')) == 1:
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+ '". }}') 
            else:
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>. }}')   
        
    else:
        return generateQuestion(listOfPred, listOfObj, index+1)
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)

  
def updateQuery(left, PosFilters, NegFilters):
    FilterQuestion =  (f"""
            select (count(*) as ?count) ?p ?o  where 
            {{
            
            {PosFilters}
             ?s ?p ?o .
            {NegFilters}
            }} 
            
            group by ?p ?o 
            ORDER BY ABS( {left} - ?count )
            Limit 10
            
            """) 
        
    return FilterQuestion

In [4]:
def pickSubject():
    #ind=randint(0,10000)
    entities = []
    query = f"""
    select distinct ?s
    where {{
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        entities.append(r['s']['value'])
        #return("<"+r["s"]["value"]+">")
    #data = pd.read_csv('abstracts.csv')
    return entities

def findAnswer(subject,p,o):
    query = f"""
        SELECT ?s ?p ?o
        WHERE {{
        ?s ?p ?o .
            FILTER(?s = {subject})
            FILTER(?p = {p})
            FILTER(?o = {o})
            }}
            """
    #print(subject)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    res = qres["results"]["bindings"]
    return (len(res))>0

In [12]:
def numberleft(PosFilters,NegFilters):
    sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-Stanford')
    query =  f"""
            select (count(?s) as ?count) where 
            {{
            {PosFilters}            
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
          }}       
            group by ?p ?o 
            ORDER BY DESC(?count )            
            
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        left= (r["count"]["value"])
        return(int(left))

In [6]:
def popentities(PosFilters,NegFilters):
    query =  f"""
            select ?s where 
            {{
            
            {PosFilters}
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
            
            }}
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    return qres["results"]["bindings"]

In [7]:
# resets are variables
def game_reset():
    PosAttr= ['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
    NegAttr= ['']
    AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    left = numberleft("".join(PosAttr), "".join(NegAttr))
    FilterQuestion =  f"""
        select (count(*) as ?count) ?p ?o  where 
        {{

        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
        ?s ?p ?o .
        }} 
        group by ?p ?o 
        ORDER BY ABS( %s - ?count )
        limit 20
        """ %((left/2))
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    return (left, qres, PosAttr, NegAttr, AttrHistory)

In [ ]:
import pandas as pd 
data1 = pd.read_csv('Abstracts-186.csv')
data1 = data1.drop('Unnamed: 0', axis=1)

data1

In [ ]:
import random
random.seed(10)
random.sample(pickSubject(), 500)

In [8]:
import random
import pandas as pd
random.seed(10)
wincount = 0
subjects_lost = []
gamesplayed = 0
lostbybreak = 0
breakskip = 0
subjects = random.sample(pickSubject(), 500)
print(subjects)
totalgames = len(subjects)
results = []
questionhistory = []

['http://yago-knowledge.org/resource/Helena_Bonham_Carter', 'http://yago-knowledge.org/resource/Eileen_Atkins', 'http://yago-knowledge.org/resource/Ellie_Kemper', 'http://yago-knowledge.org/resource/Youth_in_Revolt_(film)', 'http://yago-knowledge.org/resource/Airport_(1970_film)', 'http://yago-knowledge.org/resource/Mozambique', 'http://yago-knowledge.org/resource/Macalester_College', 'http://yago-knowledge.org/resource/KLM', 'http://yago-knowledge.org/resource/Pensacola,_Florida', 'http://yago-knowledge.org/resource/Telemundo', 'http://yago-knowledge.org/resource/Quills', 'http://yago-knowledge.org/resource/Bursaspor', 'http://yago-knowledge.org/resource/Film_noir', 'http://yago-knowledge.org/resource/Natchez,_Mississippi', 'http://yago-knowledge.org/resource/1960_Winter_Olympics', 'http://yago-knowledge.org/resource/Spencer_Tracy', 'http://yago-knowledge.org/resource/N._T._Rama_Rao', 'http://yago-knowledge.org/resource/Porto', 'http://yago-knowledge.org/resource/Peter_Capaldi', 'http

In [ ]:
gamesplayed

In [13]:
while gamesplayed < totalgames:
    # Resets all variables
    try:
        left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
        i=0
        outofguess = []
        #print(gamesplayed)
        # pick subject
        subject = "<" + subjects[gamesplayed] + ">"
        #print(subject)

        while i < 20:

            # list of Pred and Obj for questions
            listOfObj ,listOfPred = extractCountAndAttr(qres)

            # if only 1 popular entity is left break
            if left<=1:
                break

            # ask question and add filters to query
            #print('Question: %s' %(i+1))
            PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, 0, subject)

            # number of popular entities left
            left = numberleft(PosFilters, NegFilters)
            #print('left -',left)

            # run query
            sparql = SPARQLWrapper('http://localhost:7200/repositories/20Q-Stanford')
            query = updateQuery(left/2,PosFilters, NegFilters)
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            qres = sparql.queryAndConvert()

            i+=1

        # Prints answer
        if i != 20:
            for attr in popentities(PosFilters, NegFilters):
                for x in [attr['s']['value']][0:10]:
                    answer = (x.split("/")[-1])
            #print("Your Answer is: " + answer)
            if str(answer.split("/")[-1] + ">") == str(subject.split("/")[-1]):
                wincount += 1
                gamesplayed += 1
                #results.append(i)
                results.append({'Subject': subject.split("/")[-1].split('>')[0], 'Guessed-B': 'Yes', 'Questions-B': i})
                print(gamesplayed,{'Subject': subject.split("/")[-1].split('>')[0], 'Guessed-B': 'Yes', 'Questions-B': i})
                print('%s/%s' %(wincount,gamesplayed))
                print()
            else:
                gamesplayed += 1
                subjects_lost.append(subject)
                #results.append(i)
                results.append({'Subject': subject.split("/")[-1].split('>')[0], 'Guessed-B': 'No', 'Questions-B': i})
                print(gamesplayed,{'Subject': subject.split("/")[-1].split('>')[0], 'Guessed-B': 'No', 'Questions-B': i})
                print('%s/%s' %(wincount,gamesplayed))
                print()

        # ran out of questions
        else:
            #print('Final Question:')
            for j in popentities(PosFilters, NegFilters):
                    outofguess.append(f'{j["s"]["value"].split("/")[-1]}')
            guess = random.choice(outofguess)
            #print("Is it " + guess + "?")
            if str(subject.split("/")[-1]) == str(guess + ">"):
                wincount += 1
                gamesplayed += 1
                results.append({'Subject': subject.split("/")[-1].split('>')[0], 'Guessed-B': 'Yes', 'Questions-B': 20})
                #results.append(20)
                print(gamesplayed,{'Subject': subject.split("/")[-1].split('>')[0], 'Guessed': 'Yes', 'Questions-B': 20})
                print('%s/%s' %(wincount,gamesplayed))
                print()
            else:
                gamesplayed += 1
                subjects_lost.append(subject)
                results.append({'Subject': subject.split("/")[-1].split('>')[0],'Guessed-B': 'No', 'Questions-B': 20})
                #results.append(20)
                print(gamesplayed, {'Subject': subject.split("/")[-1].split('>')[0],'Guessed': 'No', 'Questions-B': 20})
                print('%s/%s' %(wincount,gamesplayed))
                print()
        breakskip = 0
        #print(AttrHistory)
        questionhistory.append(AttrHistory)
    except KeyboardInterrupt:
        break
    except:
        continue
            
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")

338 {'Subject': 'Carl_Friedrich_Gauss', 'Guessed-B': 'Yes', 'Questions-B': 16}
274/338

339 {'Subject': 'Elektra_(2005_film)', 'Guessed-B': 'Yes', 'Questions-B': 12}
275/339

340 {'Subject': 'Lucien_Ballard', 'Guessed': 'Yes', 'Questions-B': 20}
276/340

341 {'Subject': 'Terence_Blanchard', 'Guessed-B': 'Yes', 'Questions-B': 16}
277/341

342 {'Subject': 'Lower_Saxony', 'Guessed-B': 'Yes', 'Questions-B': 15}
278/342

343 {'Subject': "Michael_O'Neill_(footballer)", 'Guessed-B': 'Yes', 'Questions-B': 13}
279/343

344 {'Subject': 'Alan_Arkin', 'Guessed-B': 'Yes', 'Questions-B': 12}
280/344

345 {'Subject': 'Dabney_Coleman', 'Guessed': 'No', 'Questions-B': 20}
280/345

346 {'Subject': 'Master_of_Fine_Arts', 'Guessed': 'No', 'Questions-B': 20}
280/346

347 {'Subject': 'Jake_Shimabukuro', 'Guessed-B': 'Yes', 'Questions-B': 12}
281/347

348 {'Subject': 'Randers_FC', 'Guessed': 'No', 'Questions-B': 20}
281/348

349 {'Subject': 'T-Pain', 'Guessed-B': 'Yes', 'Questions-B': 14}
282/349

350 {'Subj

# Result Analysis

## Top 10 Questions

In [28]:
questions = []
for subject in questionhistory:
    for question in subject:
        questions.append(question)

In [29]:
from collections import Counter
c=Counter(questions)
#print(c.most_common(11))
mostcommon = []
for i in c.most_common(25)[1:]:
    print(i)
    mostcommon.append(i[0])


('<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Person>', 500)
('<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Organization>', 314)
('<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/Place>', 214)
('<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://schema.org/CreativeWork>', 195)
('<http://schema.org/knowsLanguage> <http://yago-knowledge.org/resource/English_language>', 186)
('<http://schema.org/hasOccupation> <http://yago-knowledge.org/resource/Actor>', 186)
('<http://schema.org/nationality> <http://yago-knowledge.org/resource/United_States>', 180)
('<http://schema.org/hasOccupation> <http://yago-knowledge.org/resource/Stage_actor>', 157)
('<http://schema.org/hasOccupation> <http://yago-knowledge.org/resource/Film_producer>', 151)
('<http://schema.org/hasOccupation> <http://yago-knowledge.org/resource/Screenwriter>', 139)
('<http://schema.org/hasOccupation> <http://yago-knowledge.org/resource/Writer>', 108)
('<h

In [16]:
import pandas as pd 
#topquestions = pd.DataFrame({'None IE': mostcommon})

topquestions = pd.read_csv('TopQuestionsBigData.csv')
topquestions = topquestions.drop('Unnamed: 0', axis=1)

topquestions['Stanford'] = mostcommon

topquestions.to_csv('TopQuestionsBigData.csv')
topquestions

,None IE,ClauCy,Stanford
0,<http://www.w3.org/1999/02/22-rdf-syntax-ns#ty...,<http://www.w3.org/1999/02/22-rdf-syntax-ns#ty...,<http://www.w3.org/1999/02/22-rdf-syntax-ns#ty...
1,<http://www.w3.org/1999/02/22-rdf-syntax-ns#ty...,<http://www.w3.org/1999/02/22-rdf-syntax-ns#ty...,<http://www.w3.org/1999/02/22-rdf-syntax-ns#ty...
2,<http://www.w3.org/1999/02/22-rdf-syntax-ns#ty...,<http://www.w3.org/1999/02/22-rdf-syntax-ns#ty...,<http://www.w3.org/1999/02/22-rdf-syntax-ns#ty...
3,<http://www.w3.org/1999/02/22-rdf-syntax-ns#ty...,<http://www.w3.org/1999/02/22-rdf-syntax-ns#ty...,<http://www.w3.org/1999/02/22-rdf-syntax-ns#ty...
4,<http://schema.org/knowsLanguage> <http://yago...,<http://schema.org/knowsLanguage> <http://yago...,<http://schema.org/knowsLanguage> <http://yago...
5,<http://schema.org/hasOccupation> <http://yago...,<http://schema.org/hasOccupation> <http://yago...,<http://schema.org/hasOccupation> <http://yago...
6,<http://schema.org/nationality> <http://yago-k...,<http://schema.org/nationality> <http://yago-k...,<http://schema.org/nationality> <http://yago-k...
7,<http://schema.org/hasOccupation> <http://yago...,<http://schema.org/hasOccupation> <http://yago...,<http://schema.org/hasOccupation> <http://yago...
8,<http://schema.org/hasOccupation> <http://yago...,<http://schema.org/hasOccupation> <http://yago...,<http://schema.org/hasOccupation> <http://yago...
9,<http://schema.org/hasOccupation> <http://yago...,<http://schema.org/hasOccupation> <http://yago...,<http://schema.org/hasOccupation> <http://yago...


In [30]:
for i in topquestions['None IE']:
    i = i.split(' ')
    j = i[1].split('<')[-1]
    i = i[0].split('<')[-1]
    i = i.split('/')[-1]
    i = i.split('#')[-1]
    j = j.split('/')[-1]
    j = j.split('>')[0]
    i = i.split('>')[0]
    i = " ".join(i.split('_'))
    j = " ".join(j.split('_'))
    print(i,j)
print()
for i in topquestions['ClauCy']:
    i = i.split(' ')
    j = i[1].split('<')[-1]
    i = i[0].split('<')[-1]
    i = i.split('/')[-1]
    i = i.split('#')[-1]
    j = j.split('/')[-1]
    j = j.split('>')[0]
    i = i.split('>')[0]
    i = " ".join(i.split('_'))
    j = " ".join(j.split('_'))
    print(i,j)
print()
for i in topquestions['Stanford']:
    i = i.split(' ')
    j = i[1].split('<')[-1]
    i = i[0].split('<')[-1]
    i = i.split('/')[-1]
    i = i.split('#')[-1]
    j = j.split('/')[-1]
    j = j.split('>')[0]
    i = i.split('>')[0]
    i = " ".join(i.split('_'))
    j = " ".join(j.split('_'))
    print(i,j)
print()
for i in mostcommon:
    #print(i)
    i = str(i).split(' ')
    j = i[1].split('<')[-1]
    i = i[0].split('<')[-1]
    i = i.split('/')[-1]
    i = i.split('#')[-1]
    j = j.split('/')[-1]
    j = j.split('>')[0]
    i = i.split('>')[0]
    i = " ".join(i.split('_'))
    j = " ".join(j.split('_'))
    print(i,j)

type Person
type Organization
type Place
type CreativeWork
knowsLanguage English language
hasOccupation Actor
nationality United States
hasOccupation Stage actor
hasOccupation Screenwriter
hasOccupation Film producer

type Person
type Organization
type Place
type CreativeWork
knowsLanguage English language
hasOccupation Actor
nationality United States
hasOccupation Stage actor
hasOccupation Film producer
hasOccupation Screenwriter

type Person
type Organization
type Place
type CreativeWork
knowsLanguage English language
hasOccupation Actor
nationality United States
hasOccupation Stage actor
hasOccupation Film producer
hasOccupation Screenwriter

type Person
type Organization
type Place
type CreativeWork
knowsLanguage English language
hasOccupation Actor
nationality United States
hasOccupation Stage actor
hasOccupation Film producer
hasOccupation Screenwriter
hasOccupation Writer
countryOfOrigin United States
hasOccupation Film director
genre Action film
type Corporation
countryOfOrigin

## Number of Questions asked

In [18]:
# if literal is less than 2-3 words make a link version
import pandas as pd 
thesis_results = pd.read_csv('ThesisResultsBigData.csv')
thesis_results = thesis_results.drop('Unnamed: 0', axis=1)

thesis_results

,Subject,Guessed-B,Questions-B,Questions-C,Guessed-C
0,Helena_Bonham_Carter,Yes,16,13,Yes
1,Eileen_Atkins,Yes,11,11,Yes
2,Ellie_Kemper,No,20,20,No
3,Youth_in_Revolt_(film),Yes,18,18,Yes
4,Airport_(1970_film),No,20,20,No
...,...,...,...,...,...
495,Into_the_Storm_(2009_film),Yes,20,20,Yes
496,"Lowell,_Massachusetts",Yes,19,20,No
497,Villanova_Wildcats_men's_basketball,Yes,15,14,Yes
498,Warren_G._Harding,Yes,19,16,Yes


In [19]:
questions = []
guessed = []
for i in results:
    questions.append(i['Questions-B'])
    guessed.append(i['Guessed-B'])
questions,guessed

([12,
  11,
  18,
  18,
  20,
  13,
  14,
  17,
  20,
  20,
  15,
  13,
  13,
  16,
  12,
  12,
  11,
  20,
  12,
  17,
  11,
  20,
  14,
  14,
  14,
  15,
  11,
  12,
  14,
  20,
  14,
  17,
  17,
  20,
  15,
  18,
  14,
  20,
  20,
  14,
  14,
  18,
  12,
  20,
  20,
  14,
  15,
  16,
  18,
  17,
  14,
  20,
  20,
  17,
  17,
  15,
  14,
  14,
  14,
  20,
  12,
  15,
  20,
  20,
  14,
  20,
  15,
  14,
  14,
  12,
  14,
  20,
  20,
  20,
  13,
  11,
  13,
  13,
  17,
  16,
  17,
  12,
  14,
  15,
  12,
  19,
  11,
  13,
  18,
  13,
  12,
  13,
  17,
  20,
  14,
  15,
  13,
  14,
  16,
  20,
  16,
  13,
  20,
  18,
  14,
  13,
  14,
  20,
  15,
  18,
  13,
  12,
  15,
  12,
  15,
  18,
  20,
  13,
  17,
  14,
  16,
  20,
  13,
  14,
  14,
  15,
  12,
  10,
  15,
  19,
  11,
  16,
  13,
  20,
  20,
  16,
  14,
  18,
  13,
  20,
  12,
  20,
  17,
  20,
  12,
  20,
  20,
  12,
  14,
  15,
  20,
  20,
  20,
  13,
  13,
  13,
  20,
  13,
  17,
  14,
  20,
  13,
  19,
  14,
  16,
  14,
  15

In [20]:
#308/500 - Base
#357/500 - ClauCy
#11/21 - Stanford
#thesis_results = pd.DataFrame(results)
thesis_results['Questions-S'] = questions
thesis_results['Guessed-S'] = guessed
thesis_results.to_csv('ThesisResultsBigData.csv')
thesis_results

,Subject,Guessed-B,Questions-B,Questions-C,Guessed-C,Questions-S,Guessed-S
0,Helena_Bonham_Carter,Yes,16,13,Yes,12,Yes
1,Eileen_Atkins,Yes,11,11,Yes,11,Yes
2,Ellie_Kemper,No,20,20,No,18,Yes
3,Youth_in_Revolt_(film),Yes,18,18,Yes,18,Yes
4,Airport_(1970_film),No,20,20,No,20,No
...,...,...,...,...,...,...,...
495,Into_the_Storm_(2009_film),Yes,20,20,Yes,20,Yes
496,"Lowell,_Massachusetts",Yes,19,20,No,13,Yes
497,Villanova_Wildcats_men's_basketball,Yes,15,14,Yes,15,Yes
498,Warren_G._Harding,Yes,19,16,Yes,17,Yes


In [ ]:
correct = 0
wrong = 0
for i in thesis_results['Guessed-B']:
    if i == 'Yes':
        correct += 1
    else:
        wrong +=1
correct,wrong,correct+wrong

In [32]:
def results( results1, results2, label1, label2 ):
    score1 = 0
    score2 = 0
    tie = 0
    for res1,res2 in zip( results1, results2 ):
        if res1 == res2:
            tie += 1
        elif res1 > res2:
            score2 += 1
        else:
            score1 += 1
    
    print( '%s-%s:' %( label1.upper(), label2.upper() ) )
    print( '%s score:' %( label1 ), score1, '-%s score:' %( label2 ), score2, '-tie score:', tie ) 
    print( '%s:' %(label1), round( ( score1/( score1+score2+tie ) ) *100,2 ) )
    print( '%s:' %(label2), round( ( score2/( score1+score2+tie ) ) *100,2 ) )
    print( 'tie %:', round( ( tie/( score1+score2+tie ) ) *100,2 ) )
    if score1 > score2:
        print( '%s best' %( label1.upper() ) )
    else:
        print( '%s best' %( label2.upper() ) )
    #return [score1,score2,tie]
    return

#results( thesis_results['Questions-GPT'], thesis_results['Questions-Base'], 'gpt', 'base' )
#print()
#results( thesis_results['Questions-GPT'], thesis_results['Questions-C'], 'gpt', 'clcy' ) 
#print()
#results( thesis_results['Questions-GPT'], thesis_results['Questions-S'], 'gpt', 'stan' )
#print()
results( thesis_results['Questions-C'], thesis_results['Questions-B'], 'clcy', 'base' )
print()
results( thesis_results['Questions-S'], thesis_results['Questions-B'], 'stan', 'base' )
print()
results( thesis_results['Questions-S'], thesis_results['Questions-C'], 'stan', 'clcy' )

CLCY-BASE:
clcy score: 126 -base score: 80 -tie score: 294
clcy: 25.2
base: 16.0
tie %: 58.8
CLCY best

STAN-BASE:
stan score: 176 -base score: 95 -tie score: 229
stan: 35.2
base: 19.0
tie %: 45.8
STAN best

STAN-CLCY:
stan score: 151 -clcy score: 111 -tie score: 238
stan: 30.2
clcy: 22.2
tie %: 47.6
STAN best


# Difference in questions asked

In [33]:
def results( results1, results2, label1, label2 ):
    diff = 0
    abdiff = 0
    for res1,res2 in zip( results1, results2 ):
        diff += (res2 - res1)
        abdiff += abs(res2 - res1)
    diff = round(diff/len(results1),2)
    abdiff = round(abdiff/len(results1),2)
    
    print( '%s-%s:' %( label1.upper(), label2.upper() ) )
    print('diff:',diff)
    print('abs diff:', abdiff)
    return 

results( thesis_results['Questions-C'], thesis_results['Questions-B'], 'clcy', 'base' )
print()
results( thesis_results['Questions-S'], thesis_results['Questions-B'], 'stan', 'base' )

CLCY-BASE:
diff: 0.57
abs diff: 1.19

STAN-BASE:
diff: 0.98
abs diff: 1.64


In [40]:
def nrright(model, modelname):
    correct = 0
    wrong = 0
    for i in model:
        if i == 'Yes':
            correct += 1
        else:
            wrong +=1
    print(modelname+':')
    print('Correct:',correct)
    print('Wrong:',wrong)
    print('Percent:',str(round((correct/(correct+wrong))*100,2))+'%')
    print()
    return 
nrright( thesis_results['Guessed-B'], 'Base')
print()
nrright( thesis_results['Guessed-C'], 'ClausIE')
print()
nrright( thesis_results['Guessed-S'], 'Stanford')
print()

Base:
Correct: 308
Wrong: 192
Percent: 61.6%


ClausIE:
Correct: 357
Wrong: 143
Percent: 71.4%


Stanford:
Correct: 394
Wrong: 106
Percent: 78.8%


